In [1]:
import pandas as pd

# 데이터 컬럼이 18년, 19년, 20년 다름
# 유니온하기 위해서 필요한 컬럼만 추출하고 테이블 구조 맞추기
# cols = 시도|시군구|전체인구|전체세대수

In [30]:
# 데이터 불러오기
h_16 = pd.read_csv('data-files/2016_2가(가정생활폐기물발생및처리).csv')
h_17 = pd.read_csv('data-files/2017_2가(가정생활폐기물발생및처리).csv')
h_18 = pd.read_csv('data-files/2018_2가(가정생활폐기물발생및처리).csv')
h_19 = pd.read_csv('data-files/2019_나-2-a. (시군구) 생활(가정)폐기물 발생량.csv')
h_20 = pd.read_csv('data-files/2020_2-나-1). (시군구) 생활(가정)폐기물 발생량.csv')

print(h_18.columns)
print(h_19.columns)
print(h_20.columns)

Index(['재활용가능자원 분리배출_종이류', '재활용가능자원 분리배출_재활용잔재물', '종량제에 의한 혼합배출_가연성_음식물채소류',
       '재활용가능자원 분리배출_기타', '재활용가능자원 분리배출_전자제품', '종량제에 의한 혼합배출_가연성_플라스틱류',
       '종량제에 의한 혼합배출_불연성_소계', '종량제에 의한 혼합배출_소계', '재활용가능자원 분리배출_발포수지류', '총계',
       '재활용가능자원 분리배출_영농폐기물', '종량제에 의한 혼합배출_불연성_토사류', '재활용가능자원 분리배출_가구류',
       '재활용가능자원 분리배출_기타(배출불명 등)', '음식물류 배출(분리배출 지역일 경우)', '재활용가능자원 분리배출_형광등',
       '종량제에 의한 혼합배출_가연성_소계', '재활용가능자원 분리배출_소계', '구분_시군구', '재활용가능자원 분리배출_폐식용유',
       '종량제에 의한 혼합배출_가연성_고무피혁류', '재활용가능자원 분리배출_윤활유', '재활용가능자원 분리배출_전지류',
       '재활용가능자원 분리배출_의류', '종량제에 의한 혼합배출_가연성_기타', '재활용가능자원 분리배출_비닐류(합성수지류)',
       '종량제에 의한 혼합배출_가연성_나무류', '재활용가능자원 분리배출_고철류', '종량제에 의한 혼합배출_가연성_종이류',
       '재활용가능자원 분리배출_플라스틱류', '종량제에 의한 혼합배출_불연성_금속류', '재활용가능자원 분리배출_금속캔',
       '발생량및처리현황', '종량제에 의한 혼합배출_불연성_기타', '재활용가능자원 분리배출_유리병류',
       '종량제에 의한 혼합배출_불연성_유리류', '재활용가능자원 분리배출_타이어', '구분_시도'],
      dtype='object')
Index(['시도', '시군구', '폐기물 종류', '폐기물 종류.1', '폐기물 종류.2', '2019년 발생량', '총계_재활용',
       '총계_소각', '총계_매립

In [93]:
def NTN028_preprocessing_before_2019(data, year):

    cols_inc_tot = [c for c in data.columns if '총계' in c or '소계' in c]
    data = data.drop(columns=cols_inc_tot)

    data = data.melt(id_vars=['구분_시도','구분_시군구','발생량및처리현황'])
    pivot_df = data.pivot_table(index=['구분_시도', '구분_시군구', 'variable'], columns='발생량및처리현황', values='value', fill_value=0).reset_index()


    pivot_df['variable'] = pivot_df['variable'].apply(lambda x: f'{x}_EMPTY' if '_' not in x else x)
    pivot_df['종류'], pivot_df['하위종류'] = zip(*pivot_df['variable'].apply(lambda x: x.split('_', 1)))
    pivot_df = pivot_df.drop(columns='variable')
    # pivot_df.columns = ['구분_시도', '구분_시군구', '발생량', '매립', '소각', '재활용']
    pivot_df = pivot_df[['구분_시도', '구분_시군구', '종류', '하위종류', '발생량', '매립', '소각', '재활용']]
    pivot_df = pivot_df.reset_index(drop=True)
    pivot_df['year'] = year
    return pivot_df

df_16 = NTN028_preprocessing_before_2019(h_16, 2016)
df_17 = NTN028_preprocessing_before_2019(h_17, 2017)
df_18 = NTN028_preprocessing_before_2019(h_18, 2018)

df = pd.concat([df_16, df_17, df_18],ignore_index=True)
df.to_csv('data/2016_2018_생활폐기물배출및처리')

In [108]:
def NTN028_preprocessing_after_2019(data, year):

    list_col_2019 = ['시도', '시군구','폐기물 종류','폐기물 종류.1','폐기물 종류.2','2019년 발생량','총계_매립','총계_소각','총계_재활용']
    data =data[list_col_2019]
    data['하위종류'] = data['폐기물 종류.1']+'_'+data['폐기물 종류.2']
    data = data.drop(columns=['폐기물 종류.1', '폐기물 종류.2']) 
    data['폐기물 종류'].

    data['하위종류'] = data['하위종류'].apply(lambda x: x.replace('_EMPTY', ''))
    data['하위종류'].unique()
    data['year'] = year
    return data

NTN028_preprocessing_after_2019(h_19, 2020)['폐기물 종류'].unique()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14420\891884431.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['하위종류'] = data['폐기물 종류.1']+'_'+data['폐기물 종류.2']


array(['종량제방식 등 혼합배출', '\r재활용 \r가능자원\r 분리배출', '음식물류 폐기물 분리배출'],
      dtype=object)

In [80]:
data1 = h_19
print(data1.columns)
list_col_2019 = ['시도', '시군구','폐기물 종류','폐기물 종류.1','폐기물 종류.2','2019년 발생량','총계_매립','총계_소각','총계_재활용']
data1 =data1[list_col_2019]
data1['하위종류'] = data1['폐기물 종류.1']+'_'+data1['폐기물 종류.2']
# data1 = data1.rename(columns=['구분_시도', '구분_시군구', '종류', '하위종류', '발생량', '매립', '소각', '재활용'])      
data1['하위종류'] = data1['하위종류'].apply(lambda x: x.replace('_EMPTY', ''))
data1['하위종류'].unique()

(array(['EMPTY', '가구류', '고철류', '금속캔', '기타', '기타(배출불명 등)', '발포수지류',
        '비닐류(합성수지류)', '영농폐기물', '유리병류', '윤활유', '의류', '재활용잔재물', '전자제품',
        '전지류', '종이류', '타이어', '폐식용유', '플라스틱류', '형광등', '가연성_고무피혁류', '가연성_기타',
        '가연성_나무류', '가연성_음식물채소류', '가연성_종이류', '가연성_플라스틱류', '불연성_금속류',
        '불연성_기타', '불연성_유리류', '불연성_토사류'], dtype=object),
 array(['가연성', '불연성', '건설폐재류', '기타(배출불명등)', '폐지류', '고철류', '금속캔', '폐합성수지류',
        '폐고무류', '폐유리병류', '폐의류', '폐섬유류', '폐형광등', '폐전지류', '영농폐기물', '폐식용유',
        '폐전기전자제품', '폐가구류', '기타', '재활용 잔재물', 'EMPTY'], dtype=object),
 array(['폐지류', '폐합성수지류', '폐고무류', '폐섬유류', '음식물류 폐기물', '폐목재류', '기타', '폐금속류',
        '폐유리류', '폐토사류', '폐타일 및 도자기류', '연탄재', 'EMPTY', '종이팩', '비닐류',
        '발포수지류', 'PET병', '농약용기류', '폐비닐'], dtype=object))

In [96]:
h_19

,시도,시군구,폐기물 종류,폐기물 종류.1,폐기물 종류.2,2019년 발생량,총계_재활용,총계_소각,총계_매립,총계_기타,...,공공처리_매립,공공처리_기타,자가처리_재활용,자가처리_소각,자가처리_매립,자가처리_기타,위탁처리_재활용,위탁처리_소각,위탁처리_매립,위탁처리_기타
0,서울,종로구,종량제방식 등 혼합배출,가연성,폐지류,37.8,0.0,25.0,12.8,0.0,...,12.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,서울,종로구,종량제방식 등 혼합배출,가연성,폐합성수지류,22.2,0.0,14.7,7.5,0.0,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,서울,종로구,종량제방식 등 혼합배출,가연성,폐고무류,0.7,0.0,0.5,0.2,0.0,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,서울,종로구,종량제방식 등 혼합배출,가연성,폐섬유류,2.1,0.0,1.4,0.7,0.0,...,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,서울,종로구,종량제방식 등 혼합배출,가연성,음식물류 폐기물,2.9,0.0,1.9,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8505,강원,양양군,\r재활용 \r가능자원\r 분리배출,폐전기전자제품,EMPTY,0.4,0.4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8506,강원,양양군,\r재활용 \r가능자원\r 분리배출,폐가구류,EMPTY,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8507,강원,양양군,\r재활용 \r가능자원\r 분리배출,기타,EMPTY,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.6,0.0,0.0,0.0
8508,강원,양양군,\r재활용 \r가능자원\r 분리배출,재활용 잔재물,EMPTY,1.2,0.0,0.8,0.4,0.0,...,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
df_18

발생량및처리현황,구분_시도,구분_시군구,종류,하위종류,발생량,매립,소각,재활용,year
0,강원,강릉시,음식물류 배출(분리배출 지역일 경우),EMPTY,76.9,0.0,0.0,76.9,2018
1,강원,강릉시,재활용가능자원 분리배출,가구류,6.5,0.0,0.0,6.5,2018
2,강원,강릉시,재활용가능자원 분리배출,고철류,7.2,0.0,0.0,7.2,2018
3,강원,강릉시,재활용가능자원 분리배출,금속캔,0.1,0.0,0.0,0.1,2018
4,강원,강릉시,재활용가능자원 분리배출,기타,0.0,0.0,0.0,0.0,2018
...,...,...,...,...,...,...,...,...,...
6895,충북,충주시,종량제에 의한 혼합배출,가연성_플라스틱류,36.7,0.0,36.7,0.0,2018
6896,충북,충주시,종량제에 의한 혼합배출,불연성_금속류,0.0,0.0,0.0,0.0,2018
6897,충북,충주시,종량제에 의한 혼합배출,불연성_기타,47.4,47.4,0.0,0.0,2018
6898,충북,충주시,종량제에 의한 혼합배출,불연성_유리류,0.0,0.0,0.0,0.0,2018


In [99]:
df_17.columns

Index(['구분_시도', '구분_시군구', '종류', '하위종류', '발생량', '매립', '소각', '재활용', 'year'], dtype='object', name='발생량및처리현황')

In [98]:
h_19.head()

,시도,시군구,폐기물 종류,폐기물 종류.1,폐기물 종류.2,2019년 발생량,총계_재활용,총계_소각,총계_매립,총계_기타,...,공공처리_매립,공공처리_기타,자가처리_재활용,자가처리_소각,자가처리_매립,자가처리_기타,위탁처리_재활용,위탁처리_소각,위탁처리_매립,위탁처리_기타
0,서울,종로구,종량제방식 등 혼합배출,가연성,폐지류,37.8,0.0,25.0,12.8,0.0,...,12.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,서울,종로구,종량제방식 등 혼합배출,가연성,폐합성수지류,22.2,0.0,14.7,7.5,0.0,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,서울,종로구,종량제방식 등 혼합배출,가연성,폐고무류,0.7,0.0,0.5,0.2,0.0,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,서울,종로구,종량제방식 등 혼합배출,가연성,폐섬유류,2.1,0.0,1.4,0.7,0.0,...,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,서울,종로구,종량제방식 등 혼합배출,가연성,음식물류 폐기물,2.9,0.0,1.9,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Index(['시도', '시군구', '폐기물 종류', '폐기물 종류.1', '폐기물 종류.2', '2019년 발생량', '총계_재활용',
       '총계_소각', '총계_매립', '총계_기타', '공공처리_재활용', '공공처리_소각', '공공처리_매립', '공공처리_기타',
       '자가처리_재활용', '자가처리_소각', '자가처리_매립', '자가처리_기타', '위탁처리_재활용', '위탁처리_소각',
       '위탁처리_매립', '위탁처리_기타'],
      dtype='object')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14420\1536687287.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['하위종류'] = data1['폐기물 종류.1']+'_'+data1['폐기물 종류.2']
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14420\1536687287.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['하위종류'] = data1['하위종류'].apply(lambda x: x.replace('_EMPTY', ''))


array(['가연성_폐지류', '가연성_폐합성수지류', '가연성_폐고무류', '가연성_폐섬유류', '가연성_음식물류 폐기물',
       '가연성_폐목재류', '가연성_기타', '불연성_폐금속류', '불연성_폐유리류', '불연성_폐토사류',
       '불연성_폐타일 및 도자기류', '불연성_연탄재', '불연성_기타', '건설폐재류', '기타(배출불명등)',
       '폐지류_종이팩', '폐지류_기타', '고철류', '금속캔', '폐합성수지류_비닐류', '폐합성수지류_발포수지류',
       '폐합성수지류_PET병', '폐합성수지류_기타', '폐고무류', '폐유리병류', '폐의류', '폐섬유류', '폐형광등',
       '폐전지류', '영농폐기물_농약용기류', '영농폐기물_폐비닐', '폐식용유', '폐전기전자제품', '폐가구류',
       '기타', '재활용 잔재물', 'EMPTY'], dtype=object)

In [91]:


df=h_18.melt(id_vars = ['구분_시도','구분_시군구','발생량및처리현황'])

pivot_df = df.pivot_table(index=['구분_시도', '구분_시군구', 'variable'], columns='발생량및처리현황', values='value', fill_value=0).reset_index()

pivot_df.columns = ['구분_시도', '구분_시군구', 'variable', '발생량', '매립', '소각', '재활용']

pivot_df

,구분_시도,구분_시군구,variable,발생량,매립,소각,재활용
0,강원,강릉시,음식물류 배출(분리배출 지역일 경우),0.0,76.9,0.0,76.9
1,강원,강릉시,재활용가능자원 분리배출_가구류,0.0,6.5,0.0,6.5
2,강원,강릉시,재활용가능자원 분리배출_고철류,0.0,7.2,0.0,7.2
3,강원,강릉시,재활용가능자원 분리배출_금속캔,0.0,0.1,0.0,0.1
4,강원,강릉시,재활용가능자원 분리배출_기타,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
8045,충북,충주시,종량제에 의한 혼합배출_불연성_소계,47.4,47.4,0.0,0.0
8046,충북,충주시,종량제에 의한 혼합배출_불연성_유리류,0.0,0.0,0.0,0.0
8047,충북,충주시,종량제에 의한 혼합배출_불연성_토사류,0.0,0.0,0.0,0.0
8048,충북,충주시,종량제에 의한 혼합배출_소계,169.0,253.3,84.3,0.0


In [6]:
h_18.columns, hh.columns

(Index(['재활용가능자원 분리배출_종이류', '재활용가능자원 분리배출_재활용잔재물', '종량제에 의한 혼합배출_가연성_음식물채소류',
        '재활용가능자원 분리배출_기타', '재활용가능자원 분리배출_전자제품', '종량제에 의한 혼합배출_가연성_플라스틱류',
        '종량제에 의한 혼합배출_불연성_소계', '종량제에 의한 혼합배출_소계', '재활용가능자원 분리배출_발포수지류', '총계',
        '재활용가능자원 분리배출_영농폐기물', '종량제에 의한 혼합배출_불연성_토사류', '재활용가능자원 분리배출_가구류',
        '재활용가능자원 분리배출_기타(배출불명 등)', '음식물류 배출(분리배출 지역일 경우)', '재활용가능자원 분리배출_형광등',
        '종량제에 의한 혼합배출_가연성_소계', '재활용가능자원 분리배출_소계', '구분_시군구', '재활용가능자원 분리배출_폐식용유',
        '종량제에 의한 혼합배출_가연성_고무피혁류', '재활용가능자원 분리배출_윤활유', '재활용가능자원 분리배출_전지류',
        '재활용가능자원 분리배출_의류', '종량제에 의한 혼합배출_가연성_기타', '재활용가능자원 분리배출_비닐류(합성수지류)',
        '종량제에 의한 혼합배출_가연성_나무류', '재활용가능자원 분리배출_고철류', '종량제에 의한 혼합배출_가연성_종이류',
        '재활용가능자원 분리배출_플라스틱류', '종량제에 의한 혼합배출_불연성_금속류', '재활용가능자원 분리배출_금속캔',
        '발생량및처리현황', '종량제에 의한 혼합배출_불연성_기타', '재활용가능자원 분리배출_유리병류',
        '종량제에 의한 혼합배출_불연성_유리류', '재활용가능자원 분리배출_타이어', '구분_시도'],
       dtype='object'),
 Index(['처리업체_소계', '총계_소각', '자가처리_재활용', '자치단체_매립', '처리업체_매립', '구분_시군구',
        '자가처리

,재활용가능자원 분리배출_종이류,재활용가능자원 분리배출_재활용잔재물,종량제에 의한 혼합배출_가연성_음식물채소류,재활용가능자원 분리배출_기타,재활용가능자원 분리배출_전자제품,종량제에 의한 혼합배출_가연성_플라스틱류,종량제에 의한 혼합배출_불연성_소계,종량제에 의한 혼합배출_소계,재활용가능자원 분리배출_발포수지류,총계,...,종량제에 의한 혼합배출_가연성_종이류,재활용가능자원 분리배출_플라스틱류,종량제에 의한 혼합배출_불연성_금속류,재활용가능자원 분리배출_금속캔,발생량및처리현황,종량제에 의한 혼합배출_불연성_기타,재활용가능자원 분리배출_유리병류,종량제에 의한 혼합배출_불연성_유리류,재활용가능자원 분리배출_타이어,구분_시도
0,27.4,17.4,12.4,0.0,0.3,76.7,4.2,118.8,2.4,337.6,...,3.1,16.0,1.4,10.5,발생량,1.0,18.7,0.8,0.0,서울
1,0.0,0.0,0.1,0.0,0.0,36.3,1.7,39.7,0.0,39.7,...,0.1,0.0,0.3,0.0,매립,0.3,0.0,0.3,0.0,서울
2,0.0,0.0,12.3,0.0,0.0,40.4,2.5,79.1,0.0,79.1,...,3.0,0.0,1.1,0.0,소각,0.7,0.0,0.5,0.0,서울
3,27.4,17.4,0.0,0.0,0.3,0.0,0.0,0.0,2.4,218.8,...,0.0,16.0,0.0,10.5,재활용,0.0,18.7,0.0,0.0,서울
4,60.7,9.8,0.0,0.1,0.0,24.3,0.0,160.1,0.0,368.2,...,31.5,1.2,0.0,3.5,발생량,0.0,11.7,0.0,0.0,서울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,0.2,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,8.1,...,0.0,0.6,0.0,0.2,재활용,0.0,0.4,0.0,0.1,강원
916,1.0,0.4,1.8,0.4,0.2,5.3,8.1,31.0,0.4,55.7,...,7.3,0.5,1.6,0.3,발생량,5.5,5.3,0.8,0.2,강원
917,0.0,0.0,0.0,0.0,0.0,0.0,8.1,8.1,0.0,8.1,...,0.0,0.0,1.6,0.0,매립,5.5,0.0,0.8,0.0,강원
918,0.0,0.0,1.8,0.0,0.0,5.3,0.0,22.9,0.0,34.2,...,7.3,0.0,0.0,0.0,소각,0.0,0.0,0.0,0.0,강원


In [ ]:
def NTN015_preprocessing_after_2019(data, year):
    data = data.rename(columns=lambda x:x.replace(' ', ''))
    data = data.drop(columns=['총계','지방비_소계'])

    data = pd.melt(data,['시도', '시군구', '지방채', '기타', '국비', '차관', '2019년세입수수료_(광역도시에서만작성)'])
    data = data.rename(columns={
                    '2019년세입수수료_(광역도시에서만작성)' : '세입수수료'
                    , 'variable' : '지방비_분류'
                    , 'value' : '지방비'
                    })
    data['지방비_분류'] = data['지방비_분류'].str.split('_').str[-1]
    data['year'] = year
    return data

In [29]:
h_18.head()

,재활용가능자원 분리배출_종이류,재활용가능자원 분리배출_재활용잔재물,종량제에 의한 혼합배출_가연성_음식물채소류,재활용가능자원 분리배출_기타,재활용가능자원 분리배출_전자제품,종량제에 의한 혼합배출_가연성_플라스틱류,종량제에 의한 혼합배출_불연성_소계,종량제에 의한 혼합배출_소계,재활용가능자원 분리배출_발포수지류,총계,...,종량제에 의한 혼합배출_가연성_종이류,재활용가능자원 분리배출_플라스틱류,종량제에 의한 혼합배출_불연성_금속류,재활용가능자원 분리배출_금속캔,발생량및처리현황,종량제에 의한 혼합배출_불연성_기타,재활용가능자원 분리배출_유리병류,종량제에 의한 혼합배출_불연성_유리류,재활용가능자원 분리배출_타이어,구분_시도
0,27.4,17.4,12.4,0.0,0.3,76.7,4.2,118.8,2.4,337.6,...,3.1,16.0,1.4,10.5,발생량,1.0,18.7,0.8,0.0,서울
1,0.0,0.0,0.1,0.0,0.0,36.3,1.7,39.7,0.0,39.7,...,0.1,0.0,0.3,0.0,매립,0.3,0.0,0.3,0.0,서울
2,0.0,0.0,12.3,0.0,0.0,40.4,2.5,79.1,0.0,79.1,...,3.0,0.0,1.1,0.0,소각,0.7,0.0,0.5,0.0,서울
3,27.4,17.4,0.0,0.0,0.3,0.0,0.0,0.0,2.4,218.8,...,0.0,16.0,0.0,10.5,재활용,0.0,18.7,0.0,0.0,서울
4,60.7,9.8,0.0,0.1,0.0,24.3,0.0,160.1,0.0,368.2,...,31.5,1.2,0.0,3.5,발생량,0.0,11.7,0.0,0.0,서울


In [21]:
data = h_18
cols_inc_tot = [c for c in data.columns if '총계' in c or '소계' in c]
data = data.drop(columns=cols_inc_tot)
data.columns
data = pd.melt(data, ['구분_시도', '구분_시군구'])
data['variable'].value_counts()

재활용가능자원 분리배출_종이류           920
종량제에 의한 혼합배출_가연성_종이류       920
재활용가능자원 분리배출_타이어           920
재활용가능자원 분리배출_재활용잔재물        920
재활용가능자원 분리배출_고철류           920
재활용가능자원 분리배출_발포수지류         920
재활용가능자원 분리배출_기타(배출불명 등)    920
종량제에 의한 혼합배출_불연성_기타        920
재활용가능자원 분리배출_전지류           920
재활용가능자원 분리배출_윤활유           920
종량제에 의한 혼합배출_가연성_나무류       920
발생량및처리현황                   920
재활용가능자원 분리배출_가구류           920
음식물류 배출(분리배출 지역일 경우)       920
재활용가능자원 분리배출_금속캔           920
종량제에 의한 혼합배출_가연성_고무피혁류     920
종량제에 의한 혼합배출_불연성_토사류       920
재활용가능자원 분리배출_폐식용유          920
재활용가능자원 분리배출_의류            920
재활용가능자원 분리배출_비닐류(합성수지류)    920
종량제에 의한 혼합배출_불연성_금속류       920
종량제에 의한 혼합배출_가연성_플라스틱류     920
재활용가능자원 분리배출_유리병류          920
재활용가능자원 분리배출_기타            920
종량제에 의한 혼합배출_불연성_유리류       920
종량제에 의한 혼합배출_가연성_음식물채소류    920
종량제에 의한 혼합배출_가연성_기타        920
재활용가능자원 분리배출_영농폐기물         920
재활용가능자원 분리배출_플라스틱류         920
재활용가능자원 분리배출_전자제품          920
재활용가능자원 분리배출_형광등           920
Name: variable, dtype: int64

In [139]:
dropped_h_19 = h_19.drop(columns='2019년 발생량')
melted_19 = pd.melt(dropped_h_19, ['시도', '시군구', '폐기물 종류', '폐기물 종류.1', '폐기물 종류.2'])
melted_19['폐기물 종류.2'].unique()

array(['폐지류', '폐합성수지류', '폐고무류', '폐섬유류', '음식물류 폐기물', '폐목재류', '기타', '폐금속류',
       '폐유리류', '폐토사류', '폐타일 및 도자기류', '연탄재', 'EMPTY', '종이팩', '비닐류',
       '발포수지류', 'PET병', '농약용기류', '폐비닐'], dtype=object)

In [150]:
melted_19['폐기물 종류'].value_counts()

\r재활용 \r가능자원\r 분리배출    77280
종량제방식 등 혼합배출           55200
음식물류 폐기물 분리배출           3680
Name: 폐기물 종류, dtype: int64

In [152]:
melted_18 = pd.melt(h_18, ['구분_시도', '구분_시군구'])
split_data = melted_18['variable'].apply(lambda x: x.split('_'))
split_data = split_data.apply(lambda x: x + ['EMPTY'] * (3 - len(x)))
melted_18['폐기물 종류'], melted_18['폐기물 종류.1'], melted_18['폐기물 종류.2'] = zip(*split_data)
df_18 = melted_18.drop(columns='variable')
df_18

,구분_시도,구분_시군구,value,폐기물 종류,폐기물 종류.1,폐기물 종류.2
0,서울,종로구,27.4,재활용가능자원 분리배출,종이류,EMPTY
1,서울,종로구,0.0,재활용가능자원 분리배출,종이류,EMPTY
2,서울,종로구,0.0,재활용가능자원 분리배출,종이류,EMPTY
3,서울,종로구,27.4,재활용가능자원 분리배출,종이류,EMPTY
4,서울,중구,60.7,재활용가능자원 분리배출,종이류,EMPTY
...,...,...,...,...,...,...
33115,강원,고성군,0.1,재활용가능자원 분리배출,타이어,EMPTY
33116,강원,양양군,0.2,재활용가능자원 분리배출,타이어,EMPTY
33117,강원,양양군,0.0,재활용가능자원 분리배출,타이어,EMPTY
33118,강원,양양군,0.0,재활용가능자원 분리배출,타이어,EMPTY


In [153]:
melted_19

,시도,시군구,폐기물 종류,폐기물 종류.1,폐기물 종류.2,variable,value
0,서울,종로구,종량제방식 등 혼합배출,가연성,폐지류,총계_재활용,0.0
1,서울,종로구,종량제방식 등 혼합배출,가연성,폐합성수지류,총계_재활용,0.0
2,서울,종로구,종량제방식 등 혼합배출,가연성,폐고무류,총계_재활용,0.0
3,서울,종로구,종량제방식 등 혼합배출,가연성,폐섬유류,총계_재활용,0.0
4,서울,종로구,종량제방식 등 혼합배출,가연성,음식물류 폐기물,총계_재활용,0.0
...,...,...,...,...,...,...,...
136155,강원,양양군,\r재활용 \r가능자원\r 분리배출,폐전기전자제품,EMPTY,위탁처리_기타,0.0
136156,강원,양양군,\r재활용 \r가능자원\r 분리배출,폐가구류,EMPTY,위탁처리_기타,0.0
136157,강원,양양군,\r재활용 \r가능자원\r 분리배출,기타,EMPTY,위탁처리_기타,0.0
136158,강원,양양군,\r재활용 \r가능자원\r 분리배출,재활용 잔재물,EMPTY,위탁처리_기타,0.0


In [113]:

p2_melted = filtered_melted['variable'].apply(lambda x: x.split('_',1))
melted['범주'], melted['하위범주'] = zip(*p2_melted)

ValueError: not enough values to unpack (expected 2, got 1)